# Deploy an MLflow `PyFunc` model with Model Serving

In this notebook, learn how to deploy a custom MLflow PyFunc model to a serving endpoint. MLflow pyfunc offers greater flexibility and customization to your deployment. You can run any custom model, add preprocessing or post-processing logic, or execute any arbitrary Python code. While using the MLflow built-in flavor is recommended for optimal performance, you can use MLflow PyFunc models where more customization is required. 

## Install and import libraries 

In [3]:
import json
import warnings

import numpy as np
import pandas as pd
import requests
from sklearn.ensemble import RandomForestRegressor

import mlflow
from mlflow.models import infer_signature

warnings.filterwarnings("ignore")

In [4]:
LGBM_MODEL_NAME_PREFIX = "LGBM_model_"


## 1 - Create Some Sample Models

#### 1.1 - récupérer data

In [5]:
data = pd.read_csv("preprocess_table_2.csv")

In [11]:
data.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 1.2 - séparer les dataset


In [6]:
# Séparer les caractéristiques (X) et les étiquettes (y)
X = data.drop(columns=['TARGET'])
y = data['TARGET']


In [12]:
X.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002,0,0,0,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,1,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,1,0,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,100007,0,0,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
from sklearn.model_selection import train_test_split
# Séparation des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
from sklearn.impute import SimpleImputer
# imputation
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
# Normalisation des données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
X_train[:5]

array([[ 4.66702282e-01, -1.38748472e+00,  1.39209713e+00, ...,
         0.00000000e+00,  0.00000000e+00, -1.73836477e+00],
       [-1.16214581e+00, -1.38748472e+00, -7.18340682e-01, ...,
         0.00000000e+00,  0.00000000e+00, -7.92382123e-16],
       [ 7.84462241e-01,  7.20728657e-01, -7.18340682e-01, ...,
         0.00000000e+00,  0.00000000e+00, -7.92382123e-16],
       [ 1.72526168e+00, -1.38748472e+00,  1.39209713e+00, ...,
         0.00000000e+00,  0.00000000e+00, -7.92382123e-16],
       [-2.03266733e-02,  7.20728657e-01, -7.18340682e-01, ...,
         0.00000000e+00,  0.00000000e+00, -7.92382123e-16]],
      shape=(5, 774))

In [16]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [17]:
# Définir la fonction de coût
def cost_function(y_true, y_pred_prob, threshold, cost_fn=10, cost_fp=1):
    y_pred = (y_pred_prob >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cost = (fn * cost_fn) + (fp * cost_fp)
    return cost

In [18]:
best_params = {'learning_rate': 0.1,
 'num_leaves': 50,
 'max_depth': 10,
 'min_child_samples': 30,
 'feature_fraction': 0.8,
 'bagging_fraction': 0.8,
 'bagging_freq': 5,
 'reg_alpha': 0.5,
 'reg_lambda': 0.5,
 'scale_pos_weight': 1}

In [19]:
#fit the model
model = lgb.train(best_params, train_data, num_boost_round=100, valid_sets=[test_data])
# Infer signature of the model
signature = infer_signature(X_train, model.predict(X_train))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.400368 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 98781
[LightGBM] [Info] Number of data points in the train set: 246005, number of used features: 738
[LightGBM] [Info] Start training from score 0.080730


In [20]:
essai = 12

In [21]:
with mlflow.start_run():
        model_path = f"model_{essai}"

        # Log and register our model with signature
        mlflow.sklearn.log_model(
            model,
            model_path,
            signature=signature,
            registered_model_name=f"{LGBM_MODEL_NAME_PREFIX}{essai}",
        )
        mlflow.set_tag("essai", essai)

Successfully registered model 'LGBM_model_12'.
Created version '1' of model 'LGBM_model_12'.


In [24]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

In [25]:
 #Lister les expériences
experiments = client.search_experiments()
for experiment in experiments:
    print(f"Experiment ID: {experiment.experiment_id}, Name: {experiment.name}")

Experiment ID: 0, Name: Default


In [26]:
# Remplacez 'experiment_id' par l'ID de votre expérience
experiment_id = "0"

# Lister les runs d'une expérience spécifique
runs =client.search_runs(experiment_ids=[experiment_id])
# Print run information
for run in runs:
    print(f"Run ID: {run.info.run_id}, Status: {run.info.status}")

Run ID: e8f8f69d2c99405ebeaf5e91f26cd891, Status: FINISHED
Run ID: f6a90254920845d2a2d0526458df76f4, Status: FAILED
Run ID: 0bf5e2b2d7a742e787d997b265c74733, Status: FAILED
Run ID: 8436400d075c4ee4966bef3eba207580, Status: FAILED
Run ID: c6c995c5109d4c8bb3748b7220dca1d3, Status: FAILED
Run ID: 3981020e48a84808af72899e17be1cb7, Status: FAILED
Run ID: 40674ebd5f594c81a75bb6dcd35cafed, Status: FINISHED


In [27]:
client.get_run('e8f8f69d2c99405ebeaf5e91f26cd891')

<Run: data=<RunData: metrics={}, params={}, tags={'essai': '12',
 'mlflow.log-model.history': '[{"run_id": "e8f8f69d2c99405ebeaf5e91f26cd891", '
                             '"artifact_path": "model_12", "utc_time_created": '
                             '"2025-04-11 14:20:18.492131", "model_uuid": '
                             '"e2e995c11ea04ce1ab6b5ef489dd269e", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.11.9", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.6.1", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}}]',
 'm

#### 1.3 - Test inference on our models

In [28]:
model_name = f"{LGBM_MODEL_NAME_PREFIX}{essai}"
model_uri = f"models:/{model_name}/latest"
model = mlflow.sklearn.load_model(model_uri)


head_of_test_data = X_test[:2]
first_fitted_values = model.predict(head_of_test_data)
print(first_fitted_values)

[0.05758861 0.03454713]


In [29]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix
with mlflow.start_run():
  
    # Prédire les probabilités
    y_pred_prob = model.predict(X_test, num_iteration=model.best_iteration)
    # Calculer l'AUC
    auc = roc_auc_score(y_test, y_pred_prob)
    mlflow.log_metric("auc", auc)

    # Optimiser le seuil de décision
    thresholds = np.linspace(0, 1, 100)
    costs = [cost_function(y_test, y_pred_prob, threshold) for threshold in thresholds]
    best_threshold = thresholds[np.argmin(costs)]

    # Enregistrer le meilleur seuil
    mlflow.log_param("best_threshold", best_threshold)

    # Évaluer le modèle avec le meilleur seuil
    y_pred = (y_pred_prob >= best_threshold).astype(int)
    min_cost = min(costs)
    mlflow.log_metric("min_cost", min_cost)

    # Enregistrer le modèle avec un exemple d'entrée pour la signature
    signature = mlflow.models.infer_signature(X_train, model.predict(X_train))
    mlflow.lightgbm.log_model(model, "LGBM", signature=signature, registered_model_name='essai_valide')
    



Successfully registered model 'essai_valide'.
Created version '1' of model 'essai_valide'.


AttributeError: 'Booster' object has no attribute 'models'

## 3 - Serve our Model
To test our endpoint, let's serve our model on our local machine. 
1. Open a new shell window in the root containing `mlruns` directory e.g. the same directory you ran this notebook.
2. Ensure mlflow is installed: `pip install --upgrade mlflow scikit-learn`
3. Run the bash command printed below.

In [32]:
PORT = 5001
print(
    f"""Run the below command in a new window. You must be in the same repo as your mlruns directory and have mlflow installed...
    mlflow models serve -m "models:/essai_valide/latest" --env-manager local -p {PORT}"""
)

Run the below command in a new window. You must be in the same repo as your mlruns directory and have mlflow installed...
    mlflow models serve -m "models:/essai_valide/latest" --env-manager local -p 5001


## 4 - Query our Served Model

In [33]:
len(X.columns)

774

In [53]:
head_of_test_data = X_test[:10]

In [54]:
# Convertir en DataFrame pandas
head_of_test_df = pd.DataFrame(head_of_test_data, columns=X.columns)

In [55]:
head_of_test_df .head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,0.301139,-1.387485,1.392097,1.506780,-0.577847,0.301424,-0.251074,2.212865,-0.238349,-0.463698,...,-2.973598e-17,-4.578236e-17,0.000000,-2.544090e-17,0.000000,3.922086e-17,0.0,0.0,0.0,-7.923821e-16
1,0.367530,-1.387485,1.392097,-0.663667,-0.577847,0.128625,0.302206,0.440641,0.286249,-0.128659,...,-1.336643e-01,-1.345782e-01,-0.134673,-2.229758e-01,-0.161303,-2.843136e-01,0.0,0.0,0.0,-1.961401e+00
2,-0.566227,0.720729,-0.718341,-0.663667,-0.577847,0.560624,-0.134234,-0.405418,-0.226149,-0.045929,...,-2.973598e-17,-4.578236e-17,0.000000,-2.544090e-17,0.000000,3.922086e-17,0.0,0.0,0.0,-7.923821e-16
3,-0.232553,0.720729,-0.718341,1.506780,-0.577847,-0.216975,1.587909,0.625067,1.469644,-0.193109,...,-1.336643e-01,-1.345782e-01,-0.134673,-2.229758e-01,-0.161303,-2.843136e-01,0.0,0.0,0.0,2.053243e+00
4,0.653948,0.720729,-0.718341,-0.663667,-0.577847,-0.268815,0.051737,0.320485,0.042250,-0.150335,...,-2.973598e-17,-4.578236e-17,0.000000,-2.544090e-17,0.000000,3.922086e-17,0.0,0.0,0.0,-7.923821e-16


In [56]:
head_of_test_df.iloc[4]

SK_ID_CURR                            6.539478e-01
CODE_GENDER                           7.207287e-01
FLAG_OWN_CAR                         -7.183407e-01
FLAG_OWN_REALTY                      -6.636668e-01
CNT_CHILDREN                         -5.778468e-01
                                          ...     
CC_NAME_CONTRACT_STATUS_Signed_VAR    3.922086e-17
CC_NAME_CONTRACT_STATUS_nan_MEAN      0.000000e+00
CC_NAME_CONTRACT_STATUS_nan_SUM       0.000000e+00
CC_NAME_CONTRACT_STATUS_nan_VAR       0.000000e+00
CC_COUNT                             -7.923821e-16
Name: 4, Length: 774, dtype: float64

In [67]:
head_of_test_df.to_csv('selection_test.csv', index=False)

In [63]:
#ligne4
inference_df = head_of_test_df.iloc[[4]].to_dict(orient='split')

In [64]:
inference_df

{'index': [4],
 'columns': ['SK_ID_CURR',
  'CODE_GENDER',
  'FLAG_OWN_CAR',
  'FLAG_OWN_REALTY',
  'CNT_CHILDREN',
  'AMT_INCOME_TOTAL',
  'AMT_CREDIT',
  'AMT_ANNUITY',
  'AMT_GOODS_PRICE',
  'REGION_POPULATION_RELATIVE',
  'DAYS_BIRTH',
  'DAYS_EMPLOYED',
  'DAYS_REGISTRATION',
  'DAYS_ID_PUBLISH',
  'OWN_CAR_AGE',
  'FLAG_MOBIL',
  'FLAG_EMP_PHONE',
  'FLAG_WORK_PHONE',
  'FLAG_CONT_MOBILE',
  'FLAG_PHONE',
  'FLAG_EMAIL',
  'CNT_FAM_MEMBERS',
  'REGION_RATING_CLIENT',
  'REGION_RATING_CLIENT_W_CITY',
  'HOUR_APPR_PROCESS_START',
  'REG_REGION_NOT_LIVE_REGION',
  'REG_REGION_NOT_WORK_REGION',
  'LIVE_REGION_NOT_WORK_REGION',
  'REG_CITY_NOT_LIVE_CITY',
  'REG_CITY_NOT_WORK_CITY',
  'LIVE_CITY_NOT_WORK_CITY',
  'EXT_SOURCE_1',
  'EXT_SOURCE_2',
  'EXT_SOURCE_3',
  'APARTMENTS_AVG',
  'BASEMENTAREA_AVG',
  'YEARS_BEGINEXPLUATATION_AVG',
  'YEARS_BUILD_AVG',
  'COMMONAREA_AVG',
  'ELEVATORS_AVG',
  'ENTRANCES_AVG',
  'FLOORSMAX_AVG',
  'FLOORSMIN_AVG',
  'LANDAREA_AVG',
  'LIVINGAPART

In [65]:
params = {"nom" : "ligne 4"}

In [66]:
def score_model(pdf, params):
    headers = {"Content-Type": "application/json"}
    url = f"http://127.0.0.1:{PORT}/invocations"
    ds_dict = {"dataframe_split": pdf, "params": params}
    data_json = json.dumps(ds_dict, allow_nan=True)

    response = requests.request(method="POST", headers=headers, url=url, data=data_json)
    response.raise_for_status()

    return response.json()


print('Inference on lgbm model ',params['nom'])
print(score_model(inference_df,params))

Inference on lgbm model  ligne 4
{'predictions': [0.15306798826266996]}
